In [49]:
import os

os.environ["OPENAI_API_KEY"] = "Your API Key"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain

## Generate Real Estate Listings

In [4]:
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI()

In [5]:
data_gen_template = """
Generate CSV formatted real estate listings for {num_listings} properties in San Francisco downtown area. Come up with a 
neighborhood name, price of the properties in dollar, number of bedrooms, number of bathrooms, the size of the the house in sqft, 
a description, and a neighborhood description for each listing. Each listing will have these fields in the CSV: Neighborhood, Price, Bedrooms, Bathrooms, House Size, Description, Neighborhood Description. 
Be creative when writing the description and neighborhood description. CSV format is a must.
"""

In [6]:
num_listings = 10
data_gen_prompt = PromptTemplate(template=data_gen_template, input_variables=["num_listings"])
data_gen_query = data_gen_prompt.format(num_listings=num_listings)

In [7]:
generated_listings = llm.predict(data_gen_query)

In [8]:
# take a look at the generated listing
generated_listings

'Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description\nNob Hill,1500000,2,2,1200,"Charming 2 bedroom, 2 bathroom condo with stunning city views. Updated kitchen and spacious living room.","Nob Hill is known for its historic mansions, cable cars, and upscale boutiques."\nSoMa,1300000,3,2,1400,"Modern 3 bedroom, 2 bathroom loft with high ceilings and industrial finishes. Open floor plan perfect for entertaining.","SoMa is a vibrant neighborhood with tech offices, trendy restaurants, and art galleries."\nFinancial District,1800000,2,2.5,1600,"Luxurious 2 bedroom, 2.5 bathroom penthouse with panoramic views of the Bay Bridge and skyline. Gourmet kitchen and marble bathrooms.","The Financial District is the heart of San Francisco\'s business district, with high-rise buildings and upscale dining options."\nUnion Square,1200000,1,1,900,"Cozy 1 bedroom, 1 bathroom apartment with hardwood floors and a private balcony. Close to shopping and dining in Union Square

In [9]:
# uncomment if you recreate the listings and would like to save the data into a csv file
# with open('listings.csv', 'w', newline='') as file:
#     file.write(generated_listings)

In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


## Store Listings in a Vector Database

In [10]:
import pandas as pd
df = pd.read_csv("listings.csv")

In [11]:
pd.set_option('display.max_colwidth', None)
df.head(10)

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,Nob Hill,1500000,2,2.0,1200,"Charming 2 bedroom, 2 bathroom condo with stunning city views. Updated kitchen and spacious living room.","Nob Hill is known for its historic mansions, cable cars, and upscale boutiques."
1,SoMa,1300000,3,2.0,1400,"Modern 3 bedroom, 2 bathroom loft with high ceilings and industrial finishes. Open floor plan perfect for entertaining.","SoMa is a vibrant neighborhood with tech offices, trendy restaurants, and art galleries."
2,Financial District,1800000,2,2.5,1600,"Luxurious 2 bedroom, 2.5 bathroom penthouse with panoramic views of the Bay Bridge and skyline. Gourmet kitchen and marble bathrooms.","The Financial District is the heart of San Francisco's business district, with high-rise buildings and upscale dining options."
3,Union Square,1200000,1,1.0,900,"Cozy 1 bedroom, 1 bathroom apartment with hardwood floors and a private balcony. Close to shopping and dining in Union Square.","Union Square is a bustling commercial area with upscale department stores, theaters, and art galleries."
4,Chinatown,1000000,2,1.0,1000,"Spacious 2 bedroom, 1 bathroom condo with a modern kitchen and hardwood floors. Walking distance to Chinatown's vibrant shops and restaurants.","Chinatown is a bustling neighborhood with colorful lanterns, dim sum restaurants, and traditional Chinese architecture."
5,North Beach,1400000,3,2.0,1500,"Bright 3 bedroom, 2 bathroom townhouse with a private patio and garage parking. Close to Washington Square Park and Italian cafes.","North Beach is a lively neighborhood known for its Italian heritage, cafes, and nightlife."
6,Mission District,1100000,2,1.5,1100,"Remodeled 2 bedroom, 1.5 bathroom Victorian home with a landscaped backyard. Original details and modern amenities.","The Mission District is a diverse neighborhood with vibrant murals, hipster cafes, and trendy shops."
7,Hayes Valley,1600000,4,3.0,1800,"Spacious 4 bedroom, 3 bathroom Victorian home with a modern kitchen and backyard garden. Close to Hayes Valley's boutiques and restaurants.","Hayes Valley is a trendy neighborhood known for its upscale boutiques, hip cafes, and proximity to the arts district."
8,Tenderloin,900000,1,1.0,800,"Cozy 1 bedroom, 1 bathroom apartment with a renovated kitchen and city views. Close to theaters and nightlife in the Tenderloin.","The Tenderloin is a historic neighborhood with a mix of residential buildings, theaters, and diverse dining options."
9,Russian Hill,1700000,3,2.5,1700,"Elegant 3 bedroom, 2.5 bathroom townhouse with a private rooftop deck and views of the Golden Gate Bridge. Gourmet kitchen and luxurious master suite.","Russian Hill is a picturesque neighborhood with steep hills, historic cable cars, and stunning views of the bay."


In [12]:
loader = CSVLoader(file_path="./listings.csv")
docs = loader.load()

In [13]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

In [14]:
embeddings = OpenAIEmbeddings()

In [15]:
db = Chroma.from_documents(split_docs, embedding=embeddings)

## Build the User Preference Interface

In [46]:
questions = [
    "What Neighborhoods are you considering?", 
    "What are 3 most important things for you in choosing a property?", 
    "How many bedrooms do you need?", 
    "How manay bathromms do you need?"
]
answers = [
    "SoMa, Financial District, Russian Hill",
    "Safety, price, view, school quality",
    "2 or more.",
    "2 or more."
]

In [22]:
# properly format question and answers 
user_preferences = ""
for q, a in zip(questions, answers):
    user_preferences += f"{q} {a}\n"

In [23]:
# make sure formatting is done correctly
print(user_preferences)

What Neighborhoods are you considering? SoMa, Financial District, Russian Hill
What are 3 most important things for you in choosing a property? Safety, price, and view
How many bedrooms do you need? 2 or more.
How manay bathromms do you need? 2 or more.



## Search Based on Preferences

In [24]:
# find the 5 most relevant listings based on user preferences
similar_docs = db.similarity_search(query, k=5)

In [12]:
# make sure the results make sense
similar_docs

[Document(page_content='Neighborhood: Russian Hill\nPrice: 1400000\nBedrooms: 2\nBathrooms: 2\nHouse Size: 1600\nDescription: "Contemporary loft with exposed brick walls and high ceilings. Offers sweeping views of the Golden Gate Bridge and Alcatraz."\nNeighborhood Description: "Russian Hill is a charming neighborhood with steep hills and historic cable cars."', metadata={'row': 6, 'source': './listings.csv'}),
 Document(page_content='Neighborhood: Nob Hill\nPrice: 1500000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 2000\nDescription: "Beautifully renovated Victorian home with stunning views of the city. Features a spacious backyard and modern kitchen."\nNeighborhood Description: "Nob Hill is known for its historic architecture and upscale boutiques."', metadata={'row': 0, 'source': './listings.csv'}),
 Document(page_content='Neighborhood: Pacific Heights\nPrice: 2500000\nBedrooms: 5\nBathrooms: 4\nHouse Size: 3000\nDescription: "Elegant mansion with a grand staircase and formal dining roo

## Personalize Listing Descriptions

In [40]:
instruction = """
Based on the user preferences survey below and real estate listing in the context, augment the Description, tailoring it to resonate with the buyer’s specific preferences. 
This involves subtly emphasizing aspects of the property that align with what the buyer is looking for. Ensure that the augmentation process enhances the appeal of the listing without altering factual information.
"""

In [41]:
prompt = PromptTemplate(
    template="{instruction}\n User Preferences Survey:\n{user_preferences}\n Context: {context}",
    input_variables=["instruction", "user_preferences", "context"],
)

In [48]:
for similar_doc in similar_docs:
    query = prompt.format(instruction=instruction, user_preferences=user_preferences, context=similar_doc.page_content)
    result = llm.predict(query)
    print("Query:", query)
    print("******")
    print("Result:", result)
    print("#################################################")

Query: 
Based on the user preferences survey below and real estate listing in the context, augment the Description, tailoring it to resonate with the buyer’s specific preferences. 
This involves subtly emphasizing aspects of the property that align with what the buyer is looking for. Ensure that the augmentation process enhances the appeal of the listing without altering factual information.

 User Preferences Survey:
What Neighborhoods are you considering? SoMa, Financial District, Russian Hill
What are 3 most important things for you in choosing a property? Safety, price, and view
How many bedrooms do you need? 2 or more.
How manay bathromms do you need? 2 or more.

 Context: Neighborhood: SoMa
Price: 1300000
Bedrooms: 3
Bathrooms: 2
House Size: 1400
Description: Modern 3 bedroom, 2 bathroom loft with high ceilings and industrial finishes. Open floor plan perfect for entertaining.
Neighborhood Description: SoMa is a vibrant neighborhood with tech offices, trendy restaurants, and art 